In [120]:
from config import gkey
from config import weatherkey
from citipy import citipy
import numpy as np
import pandas as pd
import requests
import json
from pprint import pprint

In [ ]:
# Build Cities Table 
# define empty lists
citylist = []
latlist = []
longlist = []
countrycodelist = []
loopcount = 0

# define length of script
MaxCities = 5


# create dataframe 
City_df = pd.DataFrame()
header_list = ['Lattitude','Longitude']
City_df = City_df.reindex(columns = header_list)      

# lats and longs
for cities in range(1,(MaxCities +1)):
    cities_lat = round(np.random.uniform(-90, 90),4)
    latlist.append(cities_lat)
    cities_long = round(np.random.uniform(-180, 180),4)
    longlist.append(cities_long)

# append Lattitude and Longitude to DF
City_df['Lattitude'] = pd.Series(latlist)
City_df['Longitude'] = pd.Series(longlist)


# Get Cities from PyCities
for i in range(0,len(City_df['Lattitude'])):
    lat = City_df.iloc[i]['Lattitude']
    long = City_df.iloc[i]['Longitude']
    cityname = citipy.nearest_city(lat,long).city_name.title()
    citycountry = citipy.nearest_city(lat,long).country_code.upper()
#     print(f'{lat} {long} {cityname} {citycountry}')
    citylist.append(cityname)
    countrycodelist.append(citycountry)

# Append Cities to DF
City_df.insert(0,'City',citylist)
City_df.insert(1, 'Country', countrycodelist)
City_df

In [ ]:
# Edit Dataframe and add columns

City_df['Temperature C'] = ""
City_df['Temperature F'] = ""
City_df['Revised Lattitude'] = ""
City_df['Revised Longitude'] = ""
City_df['Humidity %'] = ""
City_df['Cloudiness %'] = ""
City_df['Wind Speed (mph)'] = ""
City_df

In [ ]:
# api.openweathermap.org/data/2.5/weather?q=London,uk&appid={API key}

# build URL
base_url = 'http://api.openweathermap.org/data/2.5/weather?q='
# city = "Sao Filipe"
# country = 'CV'
comma = ','
andsign = '&'


# API GET & JSON
for index, row in City_df.iterrows():
    city = row['City']
    country = row['Country']
    full_url = base_url + city + comma + country + '&appid=' + weatherkey
    response = requests.get(full_url) 
    parsed_response = response.json()
    try:
        City_df.at[index,'Temperature C'] = (parsed_response["main"]["temp_max"] - 273)
        City_df.at[index,'Temperature F'] = (parsed_response["main"]["temp_max"] * 1.8 - 457.6)
        City_df.at[index,'Revised Lattitude'] = parsed_response["coord"]["lat"]
        City_df.at[index,'Revised Longitude'] = parsed_response["coord"]["lon"]
        City_df.at[index,'Humidity %'] = parsed_response["main"]["humidity"]
        City_df.at[index,'Cloudiness %'] = parsed_response["clouds"]["all"]
        City_df.at[index,'Wind Speed (mph)'] = (parsed_response["wind"]["speed"] * 2.24)
        print('city found: ' + city + " ," + country)
    except:
        print('City ' + city + ' not found! skipping')
        next
#     print(city + " " + str(CityTemp))




In [ ]:
City_df

In [ ]:
city = "Busselton"
country = 'AU'
full_url = base_url + city + comma + country + '&appid=' + weatherkey
response = requests.get(full_url) 
parsed_response = response.json()
# CityTemp = parsed_response["main"]["temp"]
pprint(parsed_response)


In [ ]:
if 'Bluff' in City_df['City'].values:
    print('yep!')



In [127]:
# define length of script
MaxCities = 5

# create dataframe 
City_df = pd.DataFrame()
header_list = ['City','Country','Lattitude','Longitude','Temperature C','Temperature F','Humidity %','Cloudiness %','Wind Speed (mph)']
City_df = City_df.reindex(columns = header_list)
City_df['City']= City_df['City'].astype(str)
City_df['Country']= City_df['Country'].astype(str)
City_df.dtypes

City                 object
Country              object
Lattitude           float64
Longitude           float64
Temperature C       float64
Temperature F       float64
Humidity %          float64
Cloudiness %        float64
Wind Speed (mph)    float64
dtype: object

In [133]:
loopcount = 0
MaxCities = 10

while loopcount < MaxCities:
    rowlist = []
    # set random lats and longs
    rand_lat = round(np.random.uniform(-90, 90),4)
    rand_long = round(np.random.uniform(-180, 180),4)
#     print(str(rand_lat) + " " + str(rand_long))
    # get city name and counry 
    cityname = citipy.nearest_city(rand_lat,rand_long).city_name.title()
    citycountry = citipy.nearest_city(rand_lat,rand_long).country_code.upper()
#     print(cityname + " " + citycountry)
    # test if city name already in list, and restart if so
    if cityname in City_df['City'].values:
        print(f'{cityname} is a Duplicate City, Restart!')
        continue
    # generate url and hit API 
    full_url = base_url + cityname + comma + citycountry + '&appid=' + weatherkey
    response = requests.get(full_url) 
    parsed_response = response.json()
    # attempt to load data into DF, if city was found and values can be parsed 
    try:
        City_df.at[loopcount, 'City']  = cityname    
        City_df.at[loopcount, 'Country'] = citycountry
        City_df.at[loopcount, 'Temperature C'] = (parsed_response["main"]["temp_max"] - 273)
        City_df.at[loopcount, 'Temperature F'] = (parsed_response["main"]["temp_max"] * 1.8 - 457.6)
        City_df.at[loopcount, 'Lattitude'] = parsed_response["coord"]["lat"]
        City_df.at[loopcount, 'Longitude'] = parsed_response["coord"]["lon"]
        City_df.at[loopcount, 'Humidity %'] = parsed_response["main"]["humidity"]
        City_df.at[loopcount, 'Cloudiness %'] = parsed_response["clouds"]["all"]
        City_df.at[loopcount, 'Wind Speed (mph)'] = (parsed_response["wind"]["speed"] * 2.24)
        print('city found: ' + cityname + " ," + citycountry)
        loopcount = loopcount + 1
    except:
        print('City ' + cityname + ' not found! skipping!')
        
    
    
#         City_df.at[index,'Country'] = citycountry
#         City_df.at[index,'Temperature C'] = (parsed_response["main"]["temp_max"] - 273)
#         City_df.at[index,'Temperature F'] = (parsed_response["main"]["temp_max"] * 1.8 - 457.6)
#         City_df.at[index,'Lattitude'] = parsed_response["coord"]["lat"]
#         City_df.at[index,'Longitude'] = parsed_response["coord"]["lon"]
#         City_df.at[index,'Humidity %'] = parsed_response["main"]["humidity"]
#         City_df.at[index,'Cloudiness %'] = parsed_response["clouds"]["all"]
#         City_df.at[index,'Wind Speed (mph)'] = (parsed_response["wind"]["speed"] * 2.24)



    
    
City_df

city found: Punta Arenas ,CL
City Karaul not found! skipping!
city found: New Norfolk ,AU
city found: Tarata ,PE
City Taolanaro not found! skipping!
city found: Victoria ,SC
city found: Albany ,AU
city found: Poirino ,IT
city found: Mahebourg ,MU
City Mataura not found! skipping!
City Belushya Guba not found! skipping!
city found: Saskylakh ,RU
city found: Bosaso ,SO
City Mataura not found! skipping!
City Palabuhanratu not found! skipping!
city found: Ribeira Grande ,PT


,City,Country,Lattitude,Longitude,Temperature C,Temperature F,Humidity %,Cloudiness %,Wind Speed (mph)
0,Punta Arenas,CL,-53.1500,-70.9167,7.15,46.670,81.0,0.0,26.4992
1,New Norfolk,AU,-42.7826,147.0587,17.93,66.074,59.0,64.0,1.9936
2,Tarata,PE,-17.4744,-70.0328,9.04,50.072,89.0,99.0,7.6160
3,Victoria,SC,-4.6167,55.4500,27.15,82.670,78.0,20.0,4.6144
4,Albany,AU,-35.0169,117.8917,16.15,62.870,94.0,93.0,6.9216
5,Poirino,IT,44.9208,7.8469,5.15,43.070,59.0,26.0,1.0080
6,Mahebourg,MU,-20.4081,57.7000,25.15,79.070,88.0,40.0,9.2288
7,Saskylakh,RU,71.9167,114.0833,-35.61,-30.298,97.0,42.0,5.6672
8,Bosaso,SO,11.2842,49.1816,23.63,76.334,68.0,0.0,5.5552
9,Ribeira Grande,PT,38.5167,-28.7000,16.08,62.744,86.0,99.0,20.7648
